In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import pims
import numpy as np
import pandas as pd
import h5py
import sys
import pathlib
import multiprocessing as mp
import cv2 as cv
import skimage as sk
import pyximport
pyximport.install(setup_args={"include_dirs":np.get_include()}, reload_support=True)
import ridge_directed_ring_detector as ring_detector
import scipy.spatial.distance as dist

In [49]:
def ridge_hough_rings(img, params, dr_filter=3):
    ridge_hough = ring_detector.RidgeHoughTransform(cv.bitwise_not(cv.cvtColor(img, cv.COLOR_BGR2GRAY)).astype(np.float32))
    default_params = ridge_hough.params.copy()

    r = (params['rc']-params['dr'], params['rc']+params['dr'])
    ridge_hough.params['circle_thresh'] = params['circle_thresh']
    ridge_hough.params['sigma'] = params['sigma']
    ridge_hough.params['curv_thresh'] = params['curv_thresh']
    ridge_hough.params['Rmin'] = r[0]
    ridge_hough.params['Rmax'] = r[1]
    ridge_hough.params['vote_thresh'] = params['vote_thresh']
    ridge_hough.params['dr'] = params['dr']
    ridge_hough.params['eccentricity'] = params['eccentricity']

    ridge_hough.img_preprocess()
    ridge_hough.rings_detection()
    ring = ridge_hough.output['rings_subpxl']

    # filter overlapping particles
    dr = ring[:,2].mean()/dr_filter
    # find distances between all particles
    distances = dist.cdist(ring[:,:2], ring[:,:2])
    # find particles that are closer than dr and are not themselves
    x, y = np.where(distances<dr)
    d_ring = x[x!=y]
    # filter out those close particles and take average of their location
    close_rings = ring[d_ring, :]
    new_rings = []
    while close_rings.size > 0:
        close_ps = dist.cdist(close_rings[0,:2].reshape((1,2)), close_rings[:,:2])
        xp, yp = np.where(close_ps<dr)
        new_rings.append(np.mean(close_rings[yp,:], axis=0))
        close_rings = close_rings[np.delete(np.arange(close_rings.shape[0]), yp)]
    new_rings = np.array(new_rings)
    # delete double particles from ring array, and replace them with averaged locations
    ind = np.arange(ring.shape[0])
    ind = np.delete(ind, d_ring)
    ring = ring[ind,:]
    if new_rings.size > 0:
        ring = np.vstack([ring, new_rings])

#     print ring
    rings = ring
    print(rings[:,2].mean())
    print(rings.shape[0])
        
    img = sk.color.gray2rgb(img)
    if rings is not None:
        rings = rings.astype(int)
        for y, x, r in zip(rings[:,1], rings[:,0], rings[:,2]):
            # draw the center of the circle
            cv.circle(img, (y,x), int(r), (200,20,20), 1, cv.CV_AA)
            
    fig = plt.figure(figsize=(15,15))
    plt.imshow(img)
    plt.axis('off')
    plt.show()
    
    

In [187]:
p = pathlib.Path('/Users/ericdeal/Dropbox (MIT)/3_postdoc/projects/sed_transport/1_data/0_main_feed_exp_data/_2018_full_data/_data/')
i = 13
j = 5
video = pims.Video(str(list(p.glob('gb_s3_e%i/edgertronic/*.mov'%i))[j]))

ridge_hough_rings(video[101], params={'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': 0, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0})

IndexError: list index out of range

In [ ]:
## settings = {
    's2_e1': {
        'vid1': {'rc': 21, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 5, 'dr': 1, 'eccentricity': 0},
        'vid2': {'rc': 21, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 5, 'dr': 1, 'eccentricity': 0},
        'vid3': {'rc': 21, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 5, 'dr': 1, 'eccentricity': 0},
        'vid4': {'rc': 21, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 5, 'dr': 1, 'eccentricity': 0},
            },
    's2_e2': {
        'vid1': {'rc': 25, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid2': {'rc': 25, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid3': {'rc': 25, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid4': {'rc': 25, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid5': {'rc': 25, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
            },
    's2_e3': {
        'vid1': {'rc': 25, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid2': {'rc': 25, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid3': {'rc': 25, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid4': {'rc': 25, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid5': {'rc': 25, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
            }, 
    's2_e4': { 
        'vid1': {'rc': 21, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 2, 'eccentricity': 0},
        'vid2': {'rc': 21, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 2, 'eccentricity': 0},
        'vid3': {'rc': 21, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 2, 'eccentricity': 0},
            },
    's2_e5': {
        'vid1': {'rc': 21, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid2': {'rc': 21, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid3': {'rc': 21, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid4': {'rc': 21, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid5': {'rc': 21, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
            },
    's2_e6': {
        'vid1': {'rc': 20, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid2': {'rc': 20, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid3': {'rc': 20, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid4': {'rc': 20, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid5': {'rc': 20, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
            },
    's3_e1': {
        'vid1': {'rc': 17, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid2': {'rc': 17, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid3': {'rc': 17, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid4': {'rc': 17, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
            },
    's3_e2': {
        'vid1': {'rc': 18, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid2': {'rc': 18, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid3': {'rc': 18, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid4': {'rc': 18, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid5': {'rc': 18, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid6': {'rc': 18, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
            },
    's3_e3': {
        'vid1': {'rc': 17, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid2': {'rc': 17, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid3': {'rc': 17, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid4': {'rc': 17, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid5': {'rc': 17, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid6': {'rc': 17, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid7': {'rc': 17, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid8': {'rc': 17, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid9': {'rc': 17, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid10': {'rc': 17, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid11': {'rc': 17, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
            }, 
    's3_e4': {
        'vid1': {'rc': 17, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid2': {'rc': 17, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid3': {'rc': 17, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid4': {'rc': 17, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid5': {'rc': 17, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid6': {'rc': 17, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid7': {'rc': 17, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid8': {'rc': 17, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid9': {'rc': 17, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid10': {'rc': 17, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid11': {'rc': 17, 'circle_thresh': 2 * np.pi * 0.1, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
            },
    's3_e5': {
        'vid1': {'rc': 12, 'circle_thresh': 2 * np.pi * 0.18, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid2': {'rc': 12, 'circle_thresh': 2 * np.pi * 0.18, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid3': {'rc': 12, 'circle_thresh': 2 * np.pi * 0.18, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid4': {'rc': 12, 'circle_thresh': 2 * np.pi * 0.18, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid5': {'rc': 12, 'circle_thresh': 2 * np.pi * 0.18, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid6': {'rc': 12, 'circle_thresh': 2 * np.pi * 0.18, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid7': {'rc': 12, 'circle_thresh': 2 * np.pi * 0.18, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid8': {'rc': 12, 'circle_thresh': 2 * np.pi * 0.18, 'sigma': 1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
            },
    's3_e6': {
        'vid1': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.14, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid2': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.14, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid3': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.14, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid4': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.14, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid5': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.14, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid6': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.14, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid7': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.14, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid8': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.14, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
            },
    's3_e7': {
        'vid1': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid2': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid3': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},       
        'vid4': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid5': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid6': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid7': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid8': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid9': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid10': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid11': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
            },
    's3_e8': {
        'vid1': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid2': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid3': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid4': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid5': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid6': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid7': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
            },
    's3_e9': {
        'vid1': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid2': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid3': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid4': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid5': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid6': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid7': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid8': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
            }, 
    's3_e10': {
        'vid1': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid2': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid3': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid4': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid5': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid6': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid7': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid8': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
            },
    's3_e11': {
        'vid1': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid2': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid3': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
            },
    's3_e12': {
        'vid1': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid2': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid3': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid4': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid5': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
            },
    's3_e13': {
        'vid1': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid2': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid3': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid4': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
        'vid5': {'rc': 11, 'circle_thresh': 2 * np.pi * 0.15, 'sigma': .1, 'curv_thresh': -100, 'vote_thresh': 6, 'dr': 1, 'eccentricity': 0},
            },
}